# Game Recommender — Data Preprocessing

Notebook is designed to be executed once.

In [46]:
# General 
import re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Utility
from collections import Counter
from ast import literal_eval

import warnings
warnings.simplefilter('ignore')

Load in data:

In [47]:
df = pd.read_csv('datasets/new/new_games_addon.csv')
print(df.shape, '\n')
df.sample(2)

(1440, 10) 



,game_title,dev_team,genre_tags,game_summary,storyline,platforms,game_modes,themes,keywords,perspectives
265,VA-11 Hall-A: Cyberpunk Bartender Action,"['SUKEBAN', 'Ysbryd Games']","['Adventure', 'Indie', 'Simulator', 'Visual No...",Learn about daily life in a cyberpunk dystopia...,VA-11 HALL-A: Cyberpunk Bartender Action is a ...,"['Linux', 'PC (Microsoft Windows)', 'Mac', 'iO...",['Single player'],['Science fiction'],"['anime', 'cyberpunk', 'female protagonist', '...",['Text']
508,Cuphead: The Delicious Last Course,['Studio MDHR'],"['Adventure', 'Indie', 'Platform']","In Cuphead: The Delicious Last Course, Cuphead...",[],"['PC (Microsoft Windows)', 'Xbox One', 'Ninten...",[],[],[],[]


Clean data:

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   game_title    1440 non-null   object
 1   dev_team      1440 non-null   object
 2   genre_tags    1440 non-null   object
 3   game_summary  1440 non-null   object
 4   storyline     1440 non-null   object
 5   platforms     1440 non-null   object
 6   game_modes    1440 non-null   object
 7   themes        1439 non-null   object
 8   keywords      1439 non-null   object
 9   perspectives  1439 non-null   object
dtypes: object(10)
memory usage: 112.6+ KB


In [49]:
# Handle missing data
df[df.isna().any(axis=1)]

,game_title,dev_team,genre_tags,game_summary,storyline,platforms,game_modes,themes,keywords,perspectives
1017,Silent Hill 2: Enhanced Edition,The second installment in the Silent Hill seri...,['PC (Microsoft Windows)'],['Single player'],['Horror'],"['survival horror', 'fan translation - portugu...",['Third person'],NaN,NaN,NaN


In [50]:
# Filter out rows with NaN values
df = df[~df.isna().any(axis=1)]

In [51]:
# Convert list like columns to Python lists
list_like_cols = ['dev_team', 'genre_tags', 'platforms', 'game_modes', 'themes', 'keywords', 'perspectives']
for col in list_like_cols:
    df[col] = df[col].apply(literal_eval)

df.sample(2)

,game_title,dev_team,genre_tags,game_summary,storyline,platforms,game_modes,themes,keywords,perspectives
17,Grand Theft Auto V,"[Rockstar North, Rockstar Games]","[Adventure, Racing, Shooter]",Grand Theft Auto V is a vast open world game s...,[],"[PlayStation 4, Xbox One, PlayStation 5, Xbox ...","[Single player, Multiplayer, Co-operative]",[],[],[]
503,Bastion,"[Supergiant Games, WB Games]","[Adventure, Indie, RPG]",A hack-and-slash RPG featuring a reactive narr...,The game takes place in the aftermath of the C...,"[Linux, PC (Microsoft Windows), Xbox 360, Mac,...",[Single player],"[Action, Fantasy]","[post-apocalyptic, crafting, skydiving, time m...",[Bird view / Isometric]


In [52]:
# Handle duplicate titles / different editions of the same game

def remove_editions(game_title: str, dilimeter: str) -> bool:
    """
    Removes addtional editions of games already present in dataset.
    """
    if dilimeter in game_title and 'Edition' in game_title\
    or dilimeter in game_title and "Director's Cut" in game_title\
    or dilimeter in game_title and 'Deluxe' in game_title\
    or dilimeter in game_title and "Remastered" in game_title:
        title_prefix = game_title.split(dilimeter)[0]
        return title_prefix in set(df['game_title'])
    
    return False

df = df[~df['game_title'].apply(lambda x: remove_editions(x, ' - '))]
df = df[~df['game_title'].apply(lambda x: remove_editions(x, ':'))]
df = df[~df.duplicated(subset='game_summary')]  # Remove duplicate summaries
df = df[~df.duplicated(subset='game_title')]  # remove duplicate titles
df.shape

(1351, 10)

In [53]:
df = df.reset_index().drop(columns='index')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351 entries, 0 to 1350
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   game_title    1351 non-null   object
 1   dev_team      1351 non-null   object
 2   genre_tags    1351 non-null   object
 3   game_summary  1351 non-null   object
 4   storyline     1351 non-null   object
 5   platforms     1351 non-null   object
 6   game_modes    1351 non-null   object
 7   themes        1351 non-null   object
 8   keywords      1351 non-null   object
 9   perspectives  1351 non-null   object
dtypes: object(10)
memory usage: 105.7+ KB


Merge `genre_tags`, `themes`, `game_modes` and `keywords` into a single column:

In [54]:
# Since keywords contains the most tags, I'll be selecting the most common / what I think could be useful
tag_counts = Counter()
for index, row in df.iterrows():
    tag_counts.update(row['keywords'])

print('Top 20 keywords:', tag_counts.most_common(20))

Top 20 keywords: [('digital distribution', 240), ('polygonal 3d', 192), ('boss fight', 184), ('voice acting', 175), ('male protagonist', 170), ('action-adventure', 168), ('steam', 168), ('death', 157), ('sequel', 150), ('achievements', 149), ('real-time combat', 142), ('explosion', 141), ('original soundtrack release', 140), ('melee', 139), ('bloody', 139), ('human', 122), ('female protagonist', 121), ('playstation trophies', 121), ('multiple endings', 118), ('mercenary', 112)]


In [55]:
# Identify more descriptive keywords for each game
key_tags = [
    '2d', 'metroidvania', 'anime', 'jrpg', 'post-apocalyptic', 'elves',
    'character customization', 'pvp', 'multiple endings', 'gothic', 'side-scrolling',
    'difficult', 'exploration', 'turn-based', 'assassin', 'immersive', 'medieval',
    'sexual content', 'sword & sorcery', 'female protagonist', 'wizards', 'aliens'
]

for index, row in tqdm(df.iterrows(), total=len(df)):
    # Append theme / game mode tags to genre tags
    row['genre_tags'] += row['themes']
    row['genre_tags'] += row['game_modes']

    # Concat game summary and storyline
    if row['storyline'][0] != '[':  # Cell not empty
        row['game_summary'] += ' ' + row['storyline']
    
    # Filter keyword tags and append to genre tags
    for tag in key_tags:
        if tag in set(row['keywords']) or tag in row['game_summary'].lower():  # <- Check if keyword tags appear in game summary
            if tag == 'jrpg' and 'RPG' in row['genre_tags']:  # JRPG and RPG are two different genres, with different gameplay mechanics
                row['genre_tags'].remove('RPG')
            
            row['genre_tags'].append(tag)

In [56]:
# Most common genre tags
def counts(tags: list):
    genre_counts.update(tags)

genre_counts = Counter()
df['genre_tags'].apply(counts)
most_common_genres = set([tag for tag, count in genre_counts.most_common(50)])
print(most_common_genres)

{'Shooter', 'multiple endings', 'aliens', 'Single player', 'post-apocalyptic', 'Arcade', 'assassin', 'Science fiction', 'Strategy', 'RPG', 'female protagonist', 'anime', 'Tactical', 'Warfare', 'exploration', 'Racing', 'Fighting', 'Thriller', 'Drama', 'Adventure', 'Mystery', 'Historical', 'side-scrolling', 'Fantasy', 'Turn Based Strategy', 'elves', 'Split screen', 'Survival', 'Kids', 'Platform', 'Horror', 'Brawler', 'Sandbox', 'Co-operative', 'Open world', 'Stealth', 'sword & sorcery', 'immersive', 'Indie', '2d', 'Simulator', 'Point-and-Click', 'Puzzle', 'jrpg', 'difficult', 'Multiplayer', 'Action', 'turn-based', 'Visual Novel', 'Comedy'}


Since keyword tags can be incorrect I'll group games that come from a series with the same tags.

In [57]:
for index, row in tqdm(df.iterrows(), total=len(df)):
    all_game_titles = set(df[:index]['game_title'].tolist() + df[index+1:]['game_title'].tolist())
    game_series = []
    game_found = False
    
    # Identify games part of a series
    for game in all_game_titles:
        # Manual check for GTA / Elder Scrolls
        if 'Grand Theft Auto' in row['game_title'] and 'Grand Theft Auto' in game\
        or 'The Elder Scrolls' in row['game_title'] and 'The Elder Scrolls' in game\
        or row['game_title'] in game:
            game_found = True
            game_series.append(game)
    
    if game_found:
        game_series.append(row['game_title'])

    # Add new genre tags to each game in series
    series_df = df[df['game_title'].isin(game_series)]
    series_genre_tags = set()
    
    def get_genre_tags(genre_list: list): 
        for tag in genre_list:
            if tag in most_common_genres:
                series_genre_tags.add(tag)

    def overwrite_tags(genre_list: list, title: str) -> list:
        return list(series_genre_tags) if title in game_series else genre_list

    series_df['genre_tags'].apply(get_genre_tags)
    df['genre_tags'] = df.apply(lambda x: overwrite_tags(x['genre_tags'], x['game_title']), axis=1)

In [61]:
# Filter / re-order columns
df = df.drop(columns=['game_modes', 'themes', 'storyline'])
df = df.rename(columns={'genre_tags': 'primary_tags'})
df = df[['game_title', 'dev_team', 'platforms', 'primary_tags', 'keywords', 'game_summary']]
df.sample(2)

,game_title,dev_team,platforms,primary_tags,keywords,game_summary
1030,Super Mario Party,[Nintendo],[Nintendo Switch],"[Card & Board Game, Comedy, Kids, Party, Singl...","[casual, minigames, dice, donkey kong, unlocka...",Turn the tables on opponents as you race acros...
738,Quantum Break,"[Remedy Entertainment, Microsoft Studios]","[PC (Microsoft Windows), Xbox One]","[Adventure, Shooter, Action, Science fiction, ...","[helicopter, time travel, time manipulation, f...",Fix Time Before It Destroys Everything!\n\nWhe...


Save dataset:

In [62]:
df.to_csv('datasets/cleaned/new_games_CLEANED.csv', index=False)